In [142]:
import os
import numpy as np
import pandas as pd

data1 = pd.read_csv('/Users/doyoung/학교/Python_Programming_JupyterNotebook/데이터/wine_quality_1.csv', sep=';')
data2 = pd.read_csv('/Users/doyoung/학교/Python_Programming_JupyterNotebook/데이터/wine_quality_2.csv', sep=';')

In [143]:
data = pd.merge(data1,data2,on='key',how='inner')

In [144]:
non_dupli_data=data.drop_duplicates(keep='first') #중복된 데이터 제거

In [145]:
non_dupli_data2=non_dupli_data.drop(['chlorides'],axis=1)

In [146]:
non_dupli_data2.fillna(method='ffill',inplace=True) #모든 변수에 미싱벨류 없음

In [147]:
non_dupli_data2.drop(['key'],axis=1,inplace=True)

In [148]:
non_dupli_data3=pd.get_dummies(non_dupli_data2)

In [149]:
Y=non_dupli_data3['quality']
X=non_dupli_data3.drop(['quality'],axis=1)
Y=Y.astype(float)

In [150]:
def split(X, Y, test_size=0.25,random=False, random_seed=None):
    X = np.array(X)
    Y = np.array(Y)
    
    indices = np.array(range(len(X)))
    test_size_len = round(test_size * len(X))
    
    if random==True:
        if random_seed != None:
            random_generator = np.random.RandomState(seed=random_seed)
            random_generator.shuffle(indices)
            
        else:
            np.random.shuffle(indices)
            
        test_indices = indices[0:test_size_len]
        train_indices = indices[test_size_len:]
        
        X_train = X[train_indices, :]
        X_test = X[test_indices, :]
        Y_train = Y[train_indices]
        Y_test = Y[test_indices]
        
    else:
        train_indices = indices[0:(len(X)-test_size_len)]
        test_indices = indices[(len(X)-test_size_len):]
        
        X_train = X[train_indices, :]
        X_test = X[test_indices, :]
        Y_train = Y[train_indices]
        Y_test = Y[test_indices]
        
    return X_train, X_test, Y_train, Y_test

In [151]:
class MLR:
    def init(self,intercept=True):
        self.coef_=None
        self.intercept=intercept
        
    def fit(self,input_X, output_Y):
        if self.intercept==True:
            new_input_X=self.intercept_add(input_X)
            self.coef_=np.linalg.inv(new_input_X.transpose()@new_input_X)@new_input_X.transpose()@output_Y
        else:
            new_input_X=input_X
            self.coef_=np.linalg.inv(new_input_X.transposespose()@new_input_X)@new_input_X.transpose()@output_Y
            
    def intercept_add(self,input_X):
        ones=np.ones(input_X.shape[0]).reshape(-1,1)
        result=np.concatenate([input_X,ones],axis=1)
        
        return result
    
    def predict(self,X_test):
        if self.intercept==True:
            new_X_test=self.intercept_add(X_test)
            pred_test=new_X_test@self.coef_.reshape(-1,1)
        else:
            new_X_test=X_test
            pred_test=new_X_test@self.coef_.reshape(-1,1)
            
        return pred_test

In [152]:
class Mertic:
    def init(self):
        self.mse_result=0
        self.mape_result=0
        
    def MSE(self, true_y, pred_y):
        residual=true_y - pred_y
        self.mse_result=np.mean(residual**2)
        
        return self.mse_result
    
    def MAPE(self,true_y,pred_y):
        residual=true_y-pred_y
        result_mape=(np.abs((residual)/true_y))*100
        self.mape_result=np.mean(result_mape)
        
        return self.mape_result

In [157]:
X_train, X_test, Y_train, Y_test=split(X,Y,random=True,test_size=0.3,random_seed=3)

In [158]:
LR=MLR()
LR.init(intercept=True)

In [159]:
LR.fit(X_train,Y_train)
pred=LR.predict(X_test)
evaluator=Mertic()

mse=evaluator.MSE(Y_test,pred)
mape=evaluator.MAPE(Y_test,pred)

In [160]:
print('MSE 결과 : {:.1f}, MAPE 결과 : {:.1f}'.format(mse,mape))

MSE 결과 : 144.4, MAPE 결과 : 173.1
